# Quantium VI  
# Task 2
Stores 77, 86, 88 have trialed a new layout in the chips department between February 2019 and April 2019 (included). We need to understand if the layout had an impact on sales performance.  
To assess that, we select control stores, i.e. stores similar to our trial stores, against which we are testing the impact of the trial. The metrics use to assess similarity between stores and impact of the trial are:
- monthly revenue;
- monthly number of customers;
- monthly number of transactions per customer;  

**Note that control stores must have data from July 2018 to June 2019, and that data is considered in monthly batches**. Control stores are to be ranked based on a similarity metric of choice, such as correlation, or a combined quantity (such as distance in a normalised parameters space).  

One ranked, we select a few control stores per trial store, assess if trial stores have significantly different stores, and we produce some summary visualisation.

In [248]:
# Importing packages
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import spacy
import string
import statistics 

In [150]:
task1_data = pd.read_csv('data/QVI_combined_data.csv',index_col=0)
task1_data.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,BRAND,PACK_SIZE,LIFESTAGE,PREMIUM_CUSTOMER
0,2018-10-17,1,1000,1,5,natural chip compny g,2,6.0,Natural,['175'],YOUNG SINGLES/COUPLES,Premium
1,2019-05-14,1,1307,348,66,ccs nacho cheese g,3,6.3,CCs,['175'],MIDAGE SINGLES/COUPLES,Budget
2,2018-11-10,1,1307,346,96,ww original stacked chips g,2,3.8,Woolworths,['160'],MIDAGE SINGLES/COUPLES,Budget
3,2019-03-09,1,1307,347,54,ccs original g,1,2.1,CCs,['175'],MIDAGE SINGLES/COUPLES,Budget
4,2019-05-20,1,1343,383,61,smiths crinkle cut chips chicken g,2,2.9,Smiths,['170'],MIDAGE SINGLES/COUPLES,Budget


In [221]:
# selecting columns and grouping by months

analysis_data = task1_data[['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TOT_SALES']]
analysis_data['MONTH'] = analysis_data['DATE'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').month)
analysis_data['YEAR'] = analysis_data['DATE'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').year)

<ipython-input-221-08643c14f774>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['MONTH'] = analysis_data['DATE'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').month)
<ipython-input-221-08643c14f774>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['YEAR'] = analysis_data['DATE'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').year)


In [222]:
analysis_data.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TOT_SALES,MONTH,YEAR
0,2018-10-17,1,1000,6.0,10,2018
1,2019-05-14,1,1307,6.3,5,2019
2,2018-11-10,1,1307,3.8,11,2018
3,2019-03-09,1,1307,2.1,3,2019
4,2019-05-20,1,1343,2.9,5,2019


### MONTHLY REVENUE
To calculate the monthly revenue I sum the TOT_SALES field, grouped by month.

In [223]:
tot_sales_df=analysis_data[['YEAR','MONTH','STORE_NBR','TOT_SALES']].groupby(['STORE_NBR','YEAR','MONTH'],as_index=False).sum()

### MONTHLY NUMBER OF CUSTOMERS
To calculate the monthly number of customers, I calculate the total transations happening per month. The instructions do not specify that I should consider the number of unique customers, and in this way I take into account the fact that customers might do multiple customs in a month.

In [224]:
tot_sales_df['NUM_CUSTOMERS']=0

In [225]:
stores_list = analysis_data['STORE_NBR'].unique()

# NOTE: expression to be used if using STORE_NBR and MONTH as indexes
# for store in stores_list:
#     for month in range(1,13):
        
        #tot_sales_df.loc[(store,month),'NUM_CUSTOMERS'] = int(len(analysis_data.loc[analysis_data.STORE_NBR == store].loc[analysis_data.MONTH == month]))

#NOTE: expression to be used if using STORE_NBR and MONTH as categorical variables
tot_sales_df['NUM_CUSTOMERS'] = tot_sales_df.apply(lambda row: int(len(analysis_data.loc[analysis_data.STORE_NBR == row.STORE_NBR].loc[analysis_data.YEAR == row.YEAR].loc[analysis_data.MONTH == row.MONTH])), axis=1)

In [226]:
tot_sales_df.head()

,STORE_NBR,YEAR,MONTH,TOT_SALES,NUM_CUSTOMERS
0,1,2018,7,188.9,49
1,1,2018,8,168.4,41
2,1,2018,9,268.1,59
3,1,2018,10,175.4,40
4,1,2018,11,184.8,45


### MONTHLY NUMBER OF TRANSACTIONS PER CUSTOMER
This quantity represents the average number of transactions per unique customer within single months.

In [227]:
transactions_per_customer_df=analysis_data.copy()
transactions_per_customer_df['TRANSACTIONS']= 1
transactions_per_customer_df = transactions_per_customer_df[['STORE_NBR','YEAR','MONTH','LYLTY_CARD_NBR','TRANSACTIONS']].groupby(
                            ['STORE_NBR','YEAR','MONTH','LYLTY_CARD_NBR'],as_index=False).sum()

In [228]:
transactions_per_customer_df.head()

,STORE_NBR,YEAR,MONTH,LYLTY_CARD_NBR,TRANSACTIONS
0,1,2018,7,1011,1
1,1,2018,7,1027,1
2,1,2018,7,1042,1
3,1,2018,7,1055,1
4,1,2018,7,1057,1


In [229]:
transactions_per_customer_df = transactions_per_customer_df[['STORE_NBR','YEAR','MONTH','TRANSACTIONS']].groupby(
                            ['STORE_NBR','YEAR','MONTH'],as_index=False).mean()

In [230]:
transactions_per_customer_df.head()

,STORE_NBR,YEAR,MONTH,TRANSACTIONS
0,1,2018,7,1.042553
1,1,2018,8,1.000000
2,1,2018,9,1.035088
3,1,2018,10,1.025641
4,1,2018,11,1.022727


### Merging dataframe for analysis

In [231]:
#final_analysis_df = tot_sales_df.join(transactions_per_customer_df, how="inner") #pd.concat([tot_sales_df,transactions_per_customer_df], axis=1)
final_analysis_df = tot_sales_df.copy()

final_analysis_df['TRANSACTIONS'] = transactions_per_customer_df['TRANSACTIONS']

final_analysis_df.head()

,STORE_NBR,YEAR,MONTH,TOT_SALES,NUM_CUSTOMERS,TRANSACTIONS
0,1,2018,7,188.9,49,1.042553
1,1,2018,8,168.4,41,1.000000
2,1,2018,9,268.1,59,1.035088
3,1,2018,10,175.4,40,1.025641
4,1,2018,11,184.8,45,1.022727


## Identifying control stores
For shops **77**,**86**,**88**.

In [241]:
# MinMax Scaling of quantities

scaled_analysis_df = final_analysis_df.copy()

scaled_analysis_df['TOT_SALES']= scaled_analysis_df['TOT_SALES'].apply(lambda x: (x-min(scaled_analysis_df['TOT_SALES']))/max(scaled_analysis_df['TOT_SALES']))
scaled_analysis_df['NUM_CUSTOMERS']= scaled_analysis_df['NUM_CUSTOMERS'].apply(lambda x: (x-min(scaled_analysis_df['NUM_CUSTOMERS']))/max(scaled_analysis_df['NUM_CUSTOMERS']))
scaled_analysis_df['TRANSACTIONS']= scaled_analysis_df['TRANSACTIONS'].apply(lambda x: (x-min(scaled_analysis_df['TRANSACTIONS']))/max(scaled_analysis_df['TRANSACTIONS']))

scaled_analysis_df.head()

,STORE_NBR,YEAR,MONTH,TOT_SALES,NUM_CUSTOMERS,TRANSACTIONS
0,1,2018,7,0.120121,0.268156,0.028369
1,1,2018,8,0.106959,0.223464,0.000000
2,1,2018,9,0.170968,0.324022,0.023392
3,1,2018,10,0.111454,0.217877,0.017094
4,1,2018,11,0.117488,0.245810,0.015152


In [259]:
# function to calculate cumulative mean square error between stores between 7/2018 and 1/2019
months_tuples = [(2018,7),(2018,8),(2018,9),(2018,10),(2018,11),(2018,12),(2019,1)]
variables = ['TOT_SALES','NUM_CUSTOMERS','TRANSACTIONS']


def mse_stores(df,store1,store2,months_tuples,variables):
    se_list=[]
    for tup in months_tuples:
        se=0
        for var in variables:
            se += (float(df.loc[df.STORE_NBR==store1].loc[df.YEAR==tup[0]].loc[df.MONTH==tup[1]][var]) -
                  float(df.loc[df.STORE_NBR==store2].loc[df.YEAR==tup[0]].loc[df.MONTH==tup[1]][var]))**2
        se_list.append(se)
    mse = statistics.mean(se_list)
    return mse


In [261]:
stores_of_interest = [77,86,88]

In [265]:
stores_list = stores_list.tolist()

for st in stores_of_interest:
    stores_list.remove(st)

True